In [1]:
import pandas as pd

# Load the orders dataset
df_orders = pd.read_csv("orders__table.csv")

# Convert 'order_date' to datetime format for further analysis
df_orders['order_date'] = pd.to_datetime(df_orders['order_date'])

# --- Step 1: Calculate Purchase Frequency, Average Order Value, and Time Between Purchases ---

# Group by user_id to get total revenue and number of purchases per customer
customer_revenue = df_orders.groupby('user_id')['order_value'].sum()  # Total revenue per customer
purchase_frequency = df_orders.groupby('user_id')['order_id'].count()  # Number of orders per customer

# Average Purchase Value
avg_order_value = customer_revenue / purchase_frequency

# Calculate time between purchases (days) for each user
df_orders['previous_order'] = df_orders.groupby('user_id')['order_date'].shift(1)
df_orders['time_between'] = (df_orders['order_date'] - df_orders['previous_order']).dt.days
avg_time_between_purchases = df_orders.groupby('user_id')['time_between'].mean().fillna(0)  # Fill first purchase NaN with 0

# --- Step 2: CLV Calculation ---

# Average Purchase Frequency Rate (for all customers)
avg_purchase_frequency = purchase_frequency.mean()

# Placeholder for Average Customer Lifespan (set as 3 years, or calculate if churn data is available)
avg_customer_lifespan = 3

# Customer Value = Average Purchase Value * Purchase Frequency Rate * Customer Lifespan
customer_value = avg_order_value * avg_purchase_frequency * avg_customer_lifespan

# Create a DataFrame for CLV and customer behavior metrics
df_clv = pd.DataFrame({
    'user_id': customer_value.index,
    'CLV': customer_value,
    'avg_order_value': avg_order_value,
    'purchase_frequency': purchase_frequency,
    'avg_time_between_purchases': avg_time_between_purchases
}).reset_index(drop=True)

# --- Step 3: Segment Customers Based on CLV ---

# Segment customers into 5 groups based on CLV
df_clv['segment'] = pd.qcut(df_clv['CLV'], 5, labels=['Low', 'Lower-Mid', 'Mid', 'Upper-Mid', 'High'])

# --- Step 4: Analyze Behavior Based on Segments ---

# Analyze the behaviors of high CLV vs. low CLV segments
high_clv_segment = df_clv[df_clv['segment'] == 'High']
low_clv_segment = df_clv[df_clv['segment'] == 'Low']

# Print summary statistics for high CLV segment
print("High CLV Segment Summary:")
print(high_clv_segment[['avg_order_value', 'purchase_frequency', 'avg_time_between_purchases']].describe())

# Print summary statistics for low CLV segment
print("\nLow CLV Segment Summary:")
print(low_clv_segment[['avg_order_value', 'purchase_frequency', 'avg_time_between_purchases']].describe())

# Display the first few rows of the segmented data
print("\nCustomer Segments and CLV:")
print(df_clv.head())


High CLV Segment Summary:
       avg_order_value  purchase_frequency  avg_time_between_purchases
count     19998.000000        19998.000000                19998.000000
mean       3176.952903            9.382438                    0.007782
std         246.166221            3.120115                   24.746933
min        2899.451111            1.000000                 -308.000000
25%        2991.877227            7.000000                  -13.053571
50%        3109.423722            9.000000                    0.000000
75%        3293.784554           11.000000                   13.000000
max        4825.260000           23.000000                  343.000000

Low CLV Segment Summary:
       avg_order_value  purchase_frequency  avg_time_between_purchases
count     19998.000000        19998.000000                19998.000000
mean       1821.622082            9.340584                    0.014351
std         246.410298            3.101710                   25.713323
min          43.200000   

In [2]:
df_clv 

,user_id,CLV,avg_order_value,purchase_frequency,avg_time_between_purchases,segment
0,1,79816.429807,2660.255000,14,-7.153846,Upper-Mid
1,2,59740.748405,1991.139231,13,-16.916667,Low
2,3,53810.544160,1793.487500,8,-7.428571,Low
3,4,70785.993602,2359.273571,14,4.461538,Lower-Mid
4,5,81692.886217,2722.796667,9,-33.000000,Upper-Mid
...,...,...,...,...,...,...
99984,99996,78957.045275,2631.612000,10,-8.222222,Upper-Mid
99985,99997,103718.971587,3456.918750,8,-10.428571,High
99986,99998,65470.719961,2182.117273,11,16.200000,Lower-Mid
99987,99999,73821.520367,2460.446667,12,-12.818182,Mid


In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load datasets
df_clicks = pd.read_csv("clicks__data.csv")
df_views = pd.read_csv("views__data.csv")
df_orders = pd.read_csv("orders__table.csv")

# Calculate CLV if not already done
# df_clv = ... (Ensure df_clv is calculated as before)

# Calculate additional engagement metrics
# 1. Calculate total quantity purchased and average order value per user
quantity_summary = df_orders.groupby('user_id')['quantity'].sum().reset_index()
avg_order_value_summary = df_orders.groupby('user_id')['order_value'].mean().reset_index()

# 2. Calculate average feedback score if applicable (assuming feedback can be quantified)
# Here we assume feedback is categorical, you may need to apply text analysis if feedback is textual
feedback_summary = df_orders.groupby('user_id')['feedback'].apply(lambda x: x.value_counts().idxmax()).reset_index()

# Merge these metrics with CLV data
df_clv_engagement = df_clv.merge(quantity_summary, on='user_id', how='left')
df_clv_engagement = df_clv_engagement.merge(avg_order_value_summary, on='user_id', how='left')
df_clv_engagement = df_clv_engagement.merge(feedback_summary, on='user_id', how='left')

# Fill missing values
df_clv_engagement['quantity'] = df_clv_engagement['quantity'].fillna(0)
df_clv_engagement['order_value'] = df_clv_engagement['order_value'].fillna(0)
df_clv_engagement['feedback'] = df_clv_engagement['feedback'].fillna('No Feedback')

# Convert feedback to numeric if it's categorical
feedback_dummies = pd.get_dummies(df_clv_engagement['feedback'])
df_clv_engagement = pd.concat([df_clv_engagement, feedback_dummies], axis=1)
df_clv_engagement.drop('feedback', axis=1, inplace=True)

# Standardize the data
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_clv_engagement[['quantity', 'order_value'] + list(feedback_dummies.columns)])

# Add scaled features to the dataframe
df_clv_engagement[['quantity', 'order_value'] + list(feedback_dummies.columns)] = scaled_features

# Calculate correlation matrix
# Only numeric columns should be used for correlation analysis
numeric_df = df_clv_engagement.select_dtypes(include=['float64', 'int64'])
correlation_matrix = numeric_df.corr()
clv_correlations = correlation_matrix['CLV'].drop('CLV')

# Print correlations
print("Correlations with CLV:")
print(clv_correlations)

# Display first few rows of the final data with additional metrics
print(df_clv_engagement.head())


Correlations with CLV:
user_id                       0.002902
avg_order_value               1.000000
purchase_frequency            0.002165
avg_time_between_purchases    0.000595
quantity                      0.002739
order_value                   1.000000
Negative feedback             0.000544
Neutral feedback              0.000097
Positive feedback            -0.001092
Name: CLV, dtype: float64
   user_id           CLV  avg_order_value  purchase_frequency  \
0        1  79816.429807      2660.255000                  14   
1        2  59740.748405      1991.139231                  13   
2        3  53810.544160      1793.487500                   8   
3        4  70785.993602      2359.273571                  14   
4        5  81692.886217      2722.796667                   9   

   avg_time_between_purchases    segment  quantity  order_value  \
0                   -7.153846  Upper-Mid  0.254365     0.329780   
1                  -16.916667        Low  0.356303    -1.045432   
2       

In [3]:
import pandas as pd

# Load data
df_orders = pd.read_csv("orders__table.csv")
df_clicks = pd.read_csv("clicks__data.csv")
df_views = pd.read_csv("views__data.csv")

# Example merging based on user_id
df_combined = pd.merge(df_orders, df_clicks, on='user_id', how='left')
df_combined = pd.merge(df_combined, df_views, on='user_id', how='left')

# Feature Engineering
# Calculate total clicks and views per user
df_combined['total_clicks'] = df_combined.groupby('user_id')['clicks'].transform('sum')
df_combined['total_views'] = df_combined.groupby('user_id')['views'].transform('sum')

# Calculate average order value and total quantity per user
df_combined['avg_order_value'] = df_combined.groupby('user_id')['order_value'].transform('mean')
df_combined['total_quantity'] = df_combined.groupby('user_id')['quantity'].transform('sum')

# Convert categorical feedback to numerical values (one-hot encoding)
feedback_dummies = pd.get_dummies(df_combined['feedback'], prefix='feedback')
df_combined = pd.concat([df_combined, feedback_dummies], axis=1)

# Drop original categorical columns and rows with NaNs
df_combined = df_combined.drop(columns=['order_date', 'feedback'])
df_combined = df_combined.dropna()

# Analyze correlation between new features and CLV
correlation_matrix = df_combined.corr()
clv_correlations = correlation_matrix['CLV'].drop('CLV')

# Print correlations
print("Correlations with CLV (including new features):")
print(clv_correlations)

# Example feature importance using a Random Forest model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Prepare features and target variable
X = df_combined.drop(['CLV', 'user_id'], axis=1)
y = df_combined['CLV']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
print("Model Score on Test Set:", model.score(X_test, y_test))


: 

In [6]:
import pandas as pd

chunk_size = 100000  # Adjust the chunk size as needed

# Process clicks data in chunks
chunks_clicks = pd.read_csv("clicks__data.csv", chunksize=chunk_size)
for chunk in chunks_clicks:
    # Process each chunk here
    print(chunk.head())

# Similarly, process views data
chunks_views = pd.read_csv("views__data.csv", chunksize=chunk_size)
for chunk in chunks_views:
    # Process each chunk here
    print(chunk.head())


   click_id  user_id        date  clicks_count
0         0    14225  2023-09-26             1
1         1    19369  2024-09-08            11
2         2      339  2024-03-08            44
3         3    19310  2023-11-26            37
4         4    13523  2023-11-20            10
        click_id  user_id        date  clicks_count
100000    100000    76454  2023-11-09             3
100001    100001    31295  2024-08-14             5
100002    100002    35044  2023-10-31             3
100003    100003    95250  2023-10-06             0
100004    100004    18969  2024-08-06             1
        click_id  user_id        date  clicks_count
200000    200000    16272  2024-03-13            26
200001    200001    65056  2023-09-22            21
200002    200002    82719  2024-07-30             8
200003    200003    20775  2023-11-03             1
200004    200004    54687  2024-03-31            18
        click_id  user_id        date  clicks_count
300000    300000    94156  2024-02-13     

In [7]:
df_orders = pd.read_csv("orders__table.csv", dtype={'order_id': 'int32', 'user_id': 'int32', 'quantity': 'int32', 'supplier_id': 'int32'})
df_clicks = pd.read_csv("clicks__data.csv", dtype={'user_id': 'int32'})
df_views = pd.read_csv("views__data.csv", dtype={'user_id': 'int32'})


In [11]:
import pandas as pd

chunk_size = 100000  # Adjust the chunk size as needed

# Initialize a list to store aggregated chunks
aggregated_clicks_chunks = []

# Process clicks data in chunks
chunks_clicks = pd.read_csv("clicks__data.csv", chunksize=chunk_size)
for chunk in chunks_clicks:
    # Aggregate click counts by user_id
    aggregated_chunk = chunk.groupby('user_id')['clicks_count'].sum().reset_index()
    aggregated_chunk.columns = ['user_id', 'total_clicks']
    aggregated_clicks_chunks.append(aggregated_chunk)

# Combine results from all chunks
df_clicks_aggregated = pd.concat(aggregated_clicks_chunks).groupby('user_id').sum().reset_index()


In [12]:
print(df_clicks_aggregated.head())


   user_id  total_clicks
0        1           188
1        2           151
2        3           125
3        4           204
4        5            39


In [14]:
chunk_size = 100000  # Adjust the chunk size as needed

# Initialize a list to store aggregated chunks
aggregated_views_chunks = []

# Process views data in chunks
chunks_views = pd.read_csv("views__data.csv", chunksize=chunk_size)
for chunk in chunks_views:
    # Aggregate view counts by user_id
    aggregated_chunk = chunk.groupby('user_id')['app_open_count'].sum().reset_index()
    aggregated_chunk.columns = ['user_id', 'total_views']
    aggregated_views_chunks.append(aggregated_chunk)

# Combine results from all chunks
df_views_aggregated = pd.concat(aggregated_views_chunks).groupby('user_id').sum().reset_index()

In [16]:
# Initialize a list to store aggregated chunks
aggregated_orders_chunks = []

# Process orders data in chunks
chunks_orders = pd.read_csv("orders__table.csv", chunksize=chunk_size)
for chunk in chunks_orders:
    # Aggregate order values by user_id
    aggregated_chunk = chunk.groupby('user_id').agg({
        'order_value': 'sum',
        'quantity': 'sum'
    }).reset_index()
    aggregated_chunk.columns = ['user_id', 'total_order_value', 'total_quantity']
    aggregated_orders_chunks.append(aggregated_chunk)

# Combine results from all chunks
df_orders_aggregated = pd.concat(aggregated_orders_chunks).groupby('user_id').sum().reset_index()


In [17]:
# Merge aggregated clicks with aggregated orders
df_combined = pd.merge(df_orders_aggregated, df_clicks_aggregated, on='user_id', how='left')

# Merge the result with aggregated views
df_combined = pd.merge(df_combined, df_views_aggregated, on='user_id', how='left')

# Fill NaN values if necessary
df_combined.fillna(0, inplace=True)


In [18]:
# Calculate average order value per user
df_combined['avg_order_value'] = df_combined['total_order_value'] / df_combined['total_quantity']

# Example: Calculate click-to-order ratio
df_combined['click_to_order_ratio'] = df_combined['total_clicks'] / (df_combined['total_order_value'] + 1)  # Avoid division by zero

# Example: Calculate view-to-click ratio
df_combined['view_to_click_ratio'] = df_combined['total_views'] / (df_combined['total_clicks'] + 1)  # Avoid division by zero


In [19]:
df_combined.to_csv("combined_data.csv", index=False)


In [20]:
print(df_combined.describe())


             user_id  total_order_value  total_quantity  total_clicks  \
count   99989.000000       99989.000000    99989.000000  99989.000000   
mean    50001.759854       25004.073788       55.009401    117.655792   
std     28867.344313        9149.604543       19.619902     50.812032   
min         1.000000          43.200000        1.000000      0.000000   
25%     25004.000000       18489.630000       41.000000     81.000000   
50%     50002.000000       24361.880000       54.000000    113.000000   
75%     75000.000000       30824.760000       68.000000    149.000000   
max    100000.000000       70527.120000      164.000000    400.000000   

        total_views  avg_order_value  click_to_order_ratio  \
count  99989.000000     99989.000000          99989.000000   
mean     245.017282       470.251979              0.004921   
std       89.760266       134.739763              0.002739   
min        1.000000         7.200000              0.000000   
25%      181.000000       382.50

In [24]:
df_combined['CLV'] = df_combined['total_order_value']

In [25]:
df_combined

,user_id,total_order_value,total_quantity,total_clicks,total_views,avg_order_value,click_to_order_ratio,view_to_click_ratio,CLV
0,1,37243.57,60,188,427,620.726167,0.005048,2.259259,37243.57
1,2,25884.81,62,151,278,417.496935,0.005833,1.828947,25884.81
2,3,14347.90,41,125,215,349.948780,0.008711,1.706349,14347.90
3,4,33029.83,78,204,414,423.459359,0.006176,2.019512,33029.83
4,5,24505.17,42,39,123,583.456429,0.001591,3.075000,24505.17
...,...,...,...,...,...,...,...,...,...
99984,99996,26316.12,51,118,280,516.002353,0.004484,2.352941,26316.12
99985,99997,27655.35,52,81,193,531.833654,0.002929,2.353659,27655.35
99986,99998,24003.29,46,175,344,521.810652,0.007290,1.954545,24003.29
99987,99999,29525.36,71,209,346,415.850141,0.007078,1.647619,29525.36


In [26]:
correlation_matrix = df_combined.corr()
clv_correlations = correlation_matrix['CLV'].drop('CLV')
print(clv_correlations)


user_id                -0.003383
total_order_value       1.000000
total_quantity          0.767945
total_clicks            0.635752
total_views             0.746605
avg_order_value         0.277234
click_to_order_ratio   -0.214781
view_to_click_ratio    -0.070835
Name: CLV, dtype: float64
